In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re
import time
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from urllib.request import urlopen
from bs4 import BeautifulSoup
import json

### Web Scrap

In [11]:
#URL to scrap
url = "http://dl.cini.it/items/show/36030"
html = urlopen(url)
soup = BeautifulSoup(html, 'html')

pages = [i.text for i in soup.find_all('a') if 'http://dl.cini.it/oa/items/' in str(i)]
lastpage = pages[-1]

In [12]:
pages

['http://dl.cini.it/oa/items/36030/manifest.json']

In [13]:
url = pages[0]
html = urlopen(url)
soup = BeautifulSoup(html, 'html')

In [14]:
json = json.loads(soup.text)
json

{'@context': 'http://www.shared-canvas.org/ns/context.json',
 '@id': 'http://dl.cini.it/oa/items/36030/manifest.json',
 '@type': 'sc:Manifest',
 'label': "Artaserse dramma per musica di Pietro Metastasio fra gli Arcadi Artino Corasio, e poeta di S.M.C. e C. da rappresentarsi in Perugia nel Teatro de' Nobili detto del Pavone nel carnevale dell'anno 1734. Dedicato alle dame della medesima cittÃ\xa0",
 'sequences': [{'@id': 'http://dl.cini.it/oa/items/36030/sequence.json',
   '@type': 'sc:Sequence',
   'label': '',
   'canvases': [{'@id': 'http://dl.cini.it/oa/items/36030/canvas.json',
     'label': 'p. 58, p. 59',
     '@type': 'sc:Canvas',
     'width': 4292,
     'height': 3784,
     'images': [{'@id': 'http://dl.cini.it/oa/files/36030/anno.json',
       'motivation': 'sc:painting',
       '@type': 'oa:Annotation',
       'resource': {'@id': 'http://dl.cini.it/files/original/6e50c10f21cd0f1d212a6ff2871bf90b.jpg',
        '@type': 'dctypes:Image',
        'format': 'image/jpeg',
       

In [15]:
path = json['sequences'][0]['canvases'][0]['images'][0]['resource']['@id']
title = json['label']

### OCR

In [7]:
import urllib.request

urllib.request.urlretrieve(path, "temporary.jpg")

('temporary.jpg', <http.client.HTTPMessage at 0x1674068ef10>)

In [19]:
def detect_text(path):
    """Detects text in the file."""
    from google.cloud import vision
    import io
    client = vision.ImageAnnotatorClient.from_service_account_json('C:/Users/gonxh/Documents/EPFL/Master/MA3/Foundations of DH/DH - Rolandi Libretti-a52be57f8f03.json')

    with io.open(path, 'rb') as image_file:
        content = image_file.read()
    
    image = vision.Image(content=content)

    response = client.text_detection(image=image)
    texts = response.text_annotations
    print('Texts:')

    #Traiter le texte pour qu'il s'append
    for text in texts:
        print(' "{}"'.format(text.description))

    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))

In [20]:
detect_text('temporary.jpg')

Texts:
 "AT TO2
58
Artab. Ah non lagnarti:
59
ATTO III.
АТТО
Lafcia a me le querele. Oggi d' ogn, altro
Più mifero fon io.
[il mio.
Artaf. Grande è il tuo duol, ma non è lieve
Nel Cafo perig!iofo
Di noftra forte ria
Non può trovar ripofo
L'afflitto amante cor .
Tenta involare il fato
La tua, la pace mia
Coll' Idolo adorato
Oggetto al noftro amor.
SCENA PRIMA:
Parte interna della Fortezza , nella quale è
ritenuto prigione Arbace. Cancelli in pro-
fpetto. Piccolła porta a mano deftra,
per la quale fi afcende alla Regia ,
6.
Nel cafo &c.
Arbace ,e poi Artaferfe
o
SCENA X V.
Artabano .
Erchè tarda è mai la morte ,
Quando è termine al martir .
A chi vive in lieta forte,
E follecito il morir.
Perche &c.
Ab.
"P.
On pur folo una volta, e dall' affanno
S quafi mi perfi
Refpiro in libertà :
Nel fentirmi d Arbace
Giudice deftinar. Ma fuperato,
Non fi penfi al periglio:
Salvai me fteffo, or fi difenda il figlio
Così ftupifce, e cade,
Pallido, e fmorto in vifo ,
Al fulmine improvifo
L'attonito Paft

### Create dictionnary

- Utiliser comme key le titre de l'oeuvre
- créer un dictionnaire avec comme key le titre de l'oeuvre et append chaque texte ensemble
- s'assurer que la key existe sinon créer une value vide (liste)
- reformat le texte lu par google vision

### Traduction